<a href="https://colab.research.google.com/github/Nik8x/LEGO_Database_PySpark_Hive/blob/master/LEGO_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LEGO Database
**The LEGO Parts/Sets/Colors and Inventories of every official LEGO set**

LEGO is a popular brand of toy building bricks. They are often sold in sets with in order to build a specific object. Each set contains a number of parts in different shapes, sizes and colors. This database contains information on which parts are included in different LEGO sets. It was originally compiled to help people who owned some LEGO sets already figure out what other sets they could build with the pieces they had.



*   How have the size of sets changed over time?
*   What colors are associated with witch themes? Could you predict which theme a set is from just by the bricks it contains?
*   What sets have the most-used pieces in them? What sets have the rarest pieces in them?
*   Have the colors of LEGOs included in sets changed over time?



![Schema Diagram for LEGO datafiles](https://rebrickable.com/static/img/diagrams/downloads_schema_v2.png)

[Kaggle Link](https://www.kaggle.com/rtatman/lego-database#colors.csv)

[LEGO Database Download](https://rebrickable.com/downloads/)

[themes.csv ](https://rebrickable.com/media/downloads/themes.csv?1557727192.3575437)

[colors.csv ](https://rebrickable.com/media/downloads/colors.csv?1557727194.8508914)

[part_categories.csv ](https://rebrickable.com/media/downloads/part_categories.csv?1557727188.1475196)

[parts.csv](https://rebrickable.com/media/downloads/parts.csv?1557727193.5342171) 

[inventories.csv ](https://rebrickable.com/media/downloads/inventories.csv?1557727192.2342098)

[sets.csv ](https://rebrickable.com/media/downloads/sets.csv?1557727193.8908858)

[inventory_parts.csv](https://rebrickable.com/media/downloads/sets.csv?1557727193.8908858)

[inventory_sets.csv ](https://rebrickable.com/media/downloads/inventory_sets.csv?1557727194.9708922)

[part_relationships.csv ](https://rebrickable.com/media/downloads/part_relationships.csv?1557727194.0842204)

In [4]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.3.0/spark-2.3.0-bin-hadoop2.7.tgz
!tar xf spark-2.3.0-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 215.6MB 93kB/s 
     |████████████████████████████████| 204kB 39.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/8d/20/f0/b30e2024226dc112e256930dd2cd4f06d00ab053c86278dcf3
Successfully built pyspark


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.0-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

sc = spark.sparkContext

from pyspark.sql import Row

### Download Data

In [0]:
!wget https://rebrickable.com/media/downloads/themes.csv?1557727192.3575437
!wget https://rebrickable.com/media/downloads/colors.csv?1557727194.8508914
!wget https://rebrickable.com/media/downloads/part_categories.csv?1557727188.1475196
!wget https://rebrickable.com/media/downloads/parts.csv?1557727193.5342171
!wget https://rebrickable.com/media/downloads/inventories.csv?1557727192.2342098
!wget https://rebrickable.com/media/downloads/sets.csv?1557727193.8908858
!wget https://rebrickable.com/media/downloads/sets.csv?1557727193.8908858
!wget https://rebrickable.com/media/downloads/inventory_sets.csv?1557727194.9708922
!wget https://rebrickable.com/media/downloads/part_relationships.csv?1557727194.0842204  

In [3]:
!ls

'colors.csv?1557727194.8508914'		     'parts.csv?1557727193.5342171'
'inventories.csv?1557727192.2342098'	      sample_data
'inventory_sets.csv?1557727194.9708922'      'sets.csv?1557727193.8908858'
'part_categories.csv?1557727188.1475196'     'sets.csv?1557727193.8908858.1'
'part_relationships.csv?1557727194.0842204'  'themes.csv?1557727192.3575437'


In [7]:
os.listdir(os.getcwd())

['.config',
 'parts.csv?1557727193.5342171',
 'part_relationships.csv?1557727194.0842204',
 'inventories.csv?1557727192.2342098',
 'spark-2.3.0-bin-hadoop2.7',
 'inventory_sets.csv?1557727194.9708922',
 '.ipynb_checkpoints',
 'sets.csv?1557727193.8908858',
 'sets.csv?1557727193.8908858.1',
 'themes.csv?1557727192.3575437',
 'colors.csv?1557727194.8508914',
 'part_categories.csv?1557727188.1475196',
 'sample_data']

### Create Database

we can use Hive commands to see databases and tables. 

However, at this point, we do not have any database or table. We will create them below.

In [8]:
spark.sql('show databases').show()

+------------+
|databaseName|
+------------+
|     default|
+------------+



In [9]:
spark.sql('show tables').show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+



In [10]:
functions =  spark.sql('show functions').collect() # total functions in Spark.SQL
len(functions)

267

Now, let's create a database. The data we will use is LEGO Database. 

In [11]:
spark.sql('create database lego')

DataFrame[]

In [12]:
# Let's check if our database has been created.

spark.sql('show databases').show()

+------------+
|databaseName|
+------------+
|     default|
|        lego|
+------------+



### Create Tables

Now, let's create tables: in textfile format, in ORC and in AVRO format. 
But first, we have to make sure we are using the movies database by switching to it using the command below.

In [13]:
spark.sql('use lego')

DataFrame[]

The lego dataset has many fields.

In [14]:
spark.sql('create table colors \
         (id int,name string,rgb string, is_trans string) \
         row format delimited fields terminated by ","\
         stored as textfile')  

spark.sql('create table inventories \
         (idUnique int,version int,set_num int) \
         row format delimited fields terminated by ","\
         stored as textfile')  

spark.sql("create table inventory_parts\
           (inventory_id int,part_num string,color_id int,quantity int, is_spare string)\
           stored as ORC")

spark.sql("create table inventory_sets\
           (inventory_id int,set_num string,quantity int)\
           stored as ORC" ) 

spark.sql("create table part_categories\
           (id int,name string)\
           stored as AVRO") 

spark.sql("create table parts\
           ( part_num string,name string, part_cat_id int)\
           stored as AVRO") 

spark.sql("create table sets\
           ( set_num string,name string, year int, theme_id int, num_parts int)\
           stored as sequencefile") 

spark.sql("create table themes\
           (id int, name string, parent_id int)\
           stored as parquet") 

DataFrame[]

In [27]:
spark.sql('show tables').show()

+--------+---------------+-----------+
|database|      tableName|isTemporary|
+--------+---------------+-----------+
|    lego|         colors|      false|
|    lego|    inventories|      false|
|    lego|inventory_parts|      false|
|    lego| inventory_sets|      false|
|    lego|part_categories|      false|
|    lego|          parts|      false|
|    lego|           sets|      false|
|    lego|         themes|      false|
+--------+---------------+-----------+



All the tables we created above.

We can get information about a table as below. If we do not include formatted or extended in the command, we see only information about the columns. But now, we see even its location, the database and other attributes

In [28]:
spark.sql("describe formatted inventories").show(truncate = False)

+----------------------------+----------------------------------------------------------+-------+
|col_name                    |data_type                                                 |comment|
+----------------------------+----------------------------------------------------------+-------+
|idUnique                    |int                                                       |null   |
|version                     |int                                                       |null   |
|set_num                     |int                                                       |null   |
|                            |                                                          |       |
|# Detailed Table Information|                                                          |       |
|Database                    |lego                                                      |       |
|Table                       |inventories                                               |       |
|Owner              

### Load Data using Hive

Now let's load data to the movies table. We can load data from a local file system or from any hadoop supported file system. If we are loading it just one time, we do not need to include overwrite. However, if there is possiblity that we could run the code more than one time, including overwrite is important not to append the same dataset to the table again and again. Hive does not do any transformation while loading data into tables. 

In [0]:
!mv colors.csv\?1557727194.8508914 colors.csv
!mv inventories.csv\?1557727192.2342098 inventories.csv
!mv inventory_sets.csv\?1557727194.9708922 inventory_sets.csv
!mv part_categories.csv\?1557727188.1475196 part_categories.csv
!mv parts.csv\?1557727193.5342171 parts.csv
!mv sets.csv\?1557727193.8908858 sets.csv
!mv themes.csv\?1557727192.3575437 themes.csv

In [56]:
spark.sql("load data local inpath '/content/colors.csv' \
           overwrite into table colors")

spark.sql("load data local inpath '/content/inventories.csv' \
            overwrite into table inventories")

spark.sql("load data local inpath '/content/inventory_sets.csv' \
            overwrite into table inventory_sets")

spark.sql("load data local inpath '/content/part_categories.csv' \
            overwrite into table part_categories")

spark.sql("load data local inpath '/content/parts.csv' \
            overwrite into table parts")

spark.sql("load data local inpath '/content/sets.csv' \
            overwrite into table sets")

spark.sql("load data local inpath '/content/themes_1.csv' \
            overwrite into table themes")

DataFrame[]

In [65]:
spark.sql("show create table inventory_sets").show(truncate = False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|createtab_stmt                                                                                                                                                                                                                                                                                                                                                                                                  |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [67]:
spark.sql("select * from colors limit 5").show(truncate = False)

+----+---------+------+--------+
|id  |name     |rgb   |is_trans|
+----+---------+------+--------+
|null|name     |rgb   |is_trans|
|-1  |[Unknown]|0033B2|f       |
|0   |Black    |05131D|f       |
|1   |Blue     |0055BF|f       |
|2   |Green    |237841|f       |
+----+---------+------+--------+



### Load Data as Dataframe

Rather than loading the data as a bulk, we can pre-process it and create a dataframe and insert our dataframe to the table.

We can create dataframes in two ways.

* by using the Spark SQL read function such as spark.read.csv, spark.read.json, spark.read.orc, spark.read.avro, spark.read.parquet, etc.
* by reading it in as an RDD and converting it to a dataframe after pre-processing it.

Let's specify schema for the inventory_parts dataset.